[View in Colaboratory](https://colab.research.google.com/github/dingaaling/roadExtract/blob/master/cnn.ipynb)

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [1]:
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [3]:
import os, cv2, sklearn, random
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
from keras import backend as K
from keras.regularizers import l2
from keras.layers import LeakyReLU
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.utils import np_utils

Using TensorFlow backend.


In [0]:
root_dir = os.getcwd() + "/drive/road_extract/data/sample/"
image_dir = root_dir + "train_sample/"
mask_dir = root_dir + "mask_sample/"

imgs, mask_imgs = [], []


for im in os.listdir(image_dir):
  imPath = image_dir + im
  maskPath = mask_dir + im.split('_')[0] + "_mask.png"
  
  imgs.append(load_image(imPath))
  mask_imgs.append(load_gray_image(maskPath))
  

imgs = np.asarray(imgs)
mask_imgs = np.asarray(mask_imgs)

In [5]:
print(imgs.shape)
print(mask_imgs.shape)

(300, 1024, 1024, 3)
(300, 1024, 1024)


In [0]:
imgs = imgs[:200,:,:,:]
mask_imgs = mask_imgs[:200,:,:]

In [7]:
print(imgs.shape)
print(mask_imgs.shape)

(200, 1024, 1024, 3)
(200, 1024, 1024)


# Create Array of Windows and Labels

In [0]:
patch_w, patch_h, patch_stride, patch_pad = 32, 32, 32, 0
img_patches = img_crop(imgs[0],patch_w,patch_h,patch_stride,patch_pad)
for i in range(1,imgs.shape[0]):
  img_patches = np.concatenate((img_patches,img_crop(imgs[i],patch_w,patch_h,patch_stride,patch_pad)))
X = img_patches
Y = np.asarray([load_mask(mask_imgs[i],patch_w,patch_h,patch_stride,0) for i in range(mask_imgs.shape[0])])
Y = np.reshape(Y,-1)


In [1]:
print(X.shape)
print(Y.shape)
print(sum(Y))

NameError: ignored

In [0]:
Y_2d = np_utils.to_categorical(Y, num_classes=2)

In [0]:
del img_patches
del imgs
del mask_imgs

In [11]:
X_road = np.zeros((np.count_nonzero(Y_2d[:,1]),X.shape[1],X.shape[2],X.shape[3]))
Y_road_2d = np.zeros((np.count_nonzero(Y_2d[:,1]),Y_2d.shape[1]))
X_nonroad = np.zeros(((X.shape[0]-np.count_nonzero(Y_2d[:,1])),X.shape[1],X.shape[2],X.shape[3]))
Y_nonroad_2d = np.zeros(((X.shape[0]-np.count_nonzero(Y_2d[:,1])),Y_2d.shape[1]))
print(X_road.shape)
print(Y_road_2d.shape)
print(X_nonroad.shape)
print(Y_nonroad_2d.shape)
road_ind = 0
nonroad_ind = 0
for i in range(X.shape[0]):
  if Y[i]>0.5:
    X_road[road_ind,:,:,:] = X[i,:,:,:]
    Y_road_2d[road_ind,:] = Y_2d[i,:]
    road_ind = road_ind+1
  else:
    X_nonroad[nonroad_ind,:,:,:] = X[i,:,:,:]
    Y_nonroad_2d[nonroad_ind,:] = Y_2d[i,:]
    nonroad_ind = nonroad_ind+1

(8123, 32, 32, 3)
(8123, 2)
(184077, 32, 32, 3)
(184077, 2)


In [0]:
X_nonroad_sample = np.zeros(X_road.shape)
Y_2d_nonroad_sample = np.zeros(Y_road_2d.shape)
rand_ind_nonroad = random.sample(np.ndarray.tolist(np.arange(X_nonroad.shape[0])),np.count_nonzero(Y_2d[:,1]))
x_ind = 0
y_ind = 0
for ind in rand_ind_nonroad:
  X_nonroad_sample[x_ind,:,:,:] = X_nonroad[ind,:,:,:]
  Y_2d_nonroad_sample[y_ind,:] = Y_nonroad_2d[ind,:]
  x_ind = x_ind+1
  y_ind = y_ind+1

In [0]:
X_even = np.zeros((2*np.count_nonzero(Y_2d[:,1]),X.shape[1],X.shape[2],X.shape[3]))
Y_even_2d = np.zeros((2*np.count_nonzero(Y_2d[:,1]),Y_2d.shape[1]))
X_even[0:np.count_nonzero(Y_2d[:,1]),:,:,:] = X_road
Y_even_2d[0:np.count_nonzero(Y_2d[:,1]),:] = Y_road_2d
X_even[np.count_nonzero(Y_2d[:,1]):2*np.count_nonzero(Y_2d[:,1]),:,:,:] = X_nonroad_sample
Y_even_2d[np.count_nonzero(Y_2d[:,1]):2*np.count_nonzero(Y_2d[:,1]),:] = Y_2d_nonroad_sample

In [0]:
del X, Y, X_road, Y_road_2d, X_nonroad, Y_nonroad_2d

In [18]:
print(X_even.shape)
print(Y_even_2d.shape)

(16246, 32, 32, 3)
(16246, 2)


# CNN Model

In [15]:
pool_size = (2, 2)
reg = 1e-6 # L2 regularization factor (used on weights, but not biases)
nb_classes = 2

model = Sequential()

model.add(Convolution2D(64, 5, 5, # 64 5x5 filters
                        border_mode='same',
                        input_shape=(patch_w,patch_h, 3)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=pool_size, border_mode='same'))
model.add(Dropout(0.25))

model.add(Convolution2D(128, 3, 3, # 128 3x3 filters
                        border_mode='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=pool_size, border_mode='same'))
model.add(Dropout(0.25))

model.add(Convolution2D(256, 3, 3, # 256 3x3 filters
                        border_mode='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=pool_size, border_mode='same'))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, W_regularizer=l2(reg))) # Fully connected layer (128 neurons)
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.5))

model.add(Dense(nb_classes, W_regularizer=l2(reg)))
model.add(Activation('softmax'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), input_shape=(32, 32, 3..., padding="same")`
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), padding="same")`
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), padding="same")`
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), padding="same")`
/us

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 64)        4864      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 16, 16, 128)       0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 128)         0         
__________

In [0]:
adam = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=adam,loss='binary_crossentropy',metrics=['accuracy'])

In [19]:
model.fit(X_even,Y_even_2d, batch_size=128, epochs=100)

Epoch 1/100
16246/16246 [==============================] - 10s 617us/step - loss: 7.1037 - acc: 0.4943
Epoch 2/100
16246/16246 [==============================] - 6s 373us/step - loss: 6.6224 - acc: 0.4991
Epoch 3/100
16246/16246 [==============================] - 6s 372us/step - loss: 5.4265 - acc: 0.5199
Epoch 4/100
16246/16246 [==============================] - 6s 375us/step - loss: 3.3435 - acc: 0.5367
Epoch 5/100
10752/16246 [==================>...........] - ETA: 2s - loss: 2.1258 - acc: 0.5414

16246/16246 [==============================] - 6s 372us/step - loss: 2.0362 - acc: 0.5397
Epoch 6/100
16246/16246 [==============================] - 6s 375us/step - loss: 1.6701 - acc: 0.5443
Epoch 7/100
16246/16246 [==============================] - 6s 374us/step - loss: 1.4539 - acc: 0.5423
Epoch 8/100
16246/16246 [==============================] - 6s 373us/step - loss: 1.2532 - acc: 0.5590
Epoch 9/100
14592/16246 [=========================>....] - ETA: 0s - loss: 1.1730 - acc: 0.5659

16246/16246 [==============================] - 6s 372us/step - loss: 1.1637 - acc: 0.5654
Epoch 10/100
16246/16246 [==============================] - 6s 376us/step - loss: 1.0670 - acc: 0.5742
Epoch 11/100
16246/16246 [==============================] - 6s 371us/step - loss: 0.9958 - acc: 0.5780
Epoch 12/100
16246/16246 [==============================] - 6s 370us/step - loss: 0.9424 - acc: 0.5868
Epoch 13/100
14592/16246 [=========================>....] - ETA: 0s - loss: 0.9031 - acc: 0.5853

16246/16246 [==============================] - 6s 373us/step - loss: 0.9067 - acc: 0.5844
Epoch 14/100
16246/16246 [==============================] - 6s 373us/step - loss: 0.8796 - acc: 0.5857
Epoch 15/100
16246/16246 [==============================] - 6s 372us/step - loss: 0.8454 - acc: 0.5990
Epoch 16/100
16246/16246 [==============================] - 6s 372us/step - loss: 0.8072 - acc: 0.6070
Epoch 17/100
15360/16246 [===========================>..] - ETA: 0s - loss: 0.8002 - acc: 0.6053

16246/16246 [==============================] - 6s 371us/step - loss: 0.8013 - acc: 0.6045
Epoch 18/100
16246/16246 [==============================] - 6s 373us/step - loss: 0.7789 - acc: 0.6103
Epoch 19/100
16246/16246 [==============================] - 6s 377us/step - loss: 0.7521 - acc: 0.6211
Epoch 20/100
16246/16246 [==============================] - 6s 373us/step - loss: 0.7498 - acc: 0.6190
Epoch 21/100
13568/16246 [========================>.....] - ETA: 1s - loss: 0.7358 - acc: 0.6223

...

16246/16246 [==============================] - 6s 386us/step - loss: 0.5088 - acc: 0.7610
Epoch 84/100
16246/16246 [==============================] - 6s 365us/step - loss: 0.5048 - acc: 0.7577
Epoch 85/100
16246/16246 [==============================] - 6s 368us/step - loss: 0.5062 - acc: 0.7597
Epoch 86/100
16246/16246 [==============================] - 6s 371us/step - loss: 0.4992 - acc: 0.7623
Epoch 87/100
14336/16246 [=========================>....] - ETA: 0s - loss: 0.5017 - acc: 0.7619

16246/16246 [==============================] - 6s 369us/step - loss: 0.5009 - acc: 0.7632
Epoch 88/100
16246/16246 [==============================] - 6s 366us/step - loss: 0.4983 - acc: 0.7667
Epoch 89/100
16246/16246 [==============================] - 6s 368us/step - loss: 0.4953 - acc: 0.7670
Epoch 90/100
16246/16246 [==============================] - 6s 365us/step - loss: 0.4993 - acc: 0.7664
Epoch 91/100
16246/16246 [==============================] - 6s 366us/step - loss: 0.4962 - acc: 0.7713
Epoch 92/100
  640/16246 [>.............................] - ETA: 5s - loss: 0.5531 - acc: 0.7250

16246/16246 [==============================] - 6s 368us/step - loss: 0.4962 - acc: 0.7676
Epoch 93/100
16246/16246 [==============================] - 6s 366us/step - loss: 0.4950 - acc: 0.7684
Epoch 94/100
16246/16246 [==============================] - 6s 368us/step - loss: 0.4928 - acc: 0.7687
Epoch 95/100
16246/16246 [==============================] - 6s 364us/step - loss: 0.4906 - acc: 0.7676
Epoch 96/100
13952/16246 [========================>.....] - ETA: 0s - loss: 0.4904 - acc: 0.7696

16246/16246 [==============================] - 6s 364us/step - loss: 0.4914 - acc: 0.7689
Epoch 97/100
16246/16246 [==============================] - 6s 367us/step - loss: 0.4912 - acc: 0.7709
Epoch 98/100
16246/16246 [==============================] - 6s 366us/step - loss: 0.4923 - acc: 0.7688
Epoch 99/100
16246/16246 [==============================] - 6s 370us/step - loss: 0.4887 - acc: 0.7683
Epoch 100/100
15360/16246 [===========================>..] - ETA: 0s - loss: 0.4883 - acc: 0.7693

16246/16246 [==============================] - 6s 373us/step - loss: 0.4874 - acc: 0.7695


# Helper Functions

In [0]:
def img_crop(im, w, h, stride, padding):
    """ Crop an image into patches, taking into account mirror boundary conditions. """
    assert len(im.shape) == 3, 'Expected RGB image.'
    num_patches = int((im.shape[0]-w)*(im.shape[1]-h)/(stride*stride))
    np_patches = np.ndarray((num_patches,w,h,3))
    imgwidth = im.shape[0]
    half_w = int(w/2)
    half_h = int(h/2)
    imgheight = im.shape[1]
#     im = np.lib.pad(im, ((padding, padding), (padding, padding), (0,0)), 'reflect')
    ind = 0
    for i in range(int(padding+(h/2)),int(imgheight+padding-(h/2)),stride):
        for j in range(int(padding+(w/2)),int(imgwidth+padding-(w/2)),stride):
            im_patch = im[j-half_w-padding:j+half_w+padding, i-half_h-padding:i+half_h+padding, :]
            np_patches[ind,:,:,:] = im_patch
            ind = ind+1
    return np_patches
      
def create_patches(X, patch_size, stride, padding):
    img_patches = np.asarray([img_crop(X[i], patch_size, patch_size, stride, padding) for i in range(X.shape[0])])
    # Linearize list
    img_patches = img_patches.reshape(-1, img_patches.shape[2], img_patches.shape[3], img_patches.shape[4])
    return img_patches
  
def load_mask(img,w,h,stride,padding):
    list_patches = []
    imgwidth = img.shape[0]
    imgheight = img.shape[1]
    for i in range(int(padding+(h/2)),int(imgheight+padding-(h/2)),stride):
      for j in range(int(padding+(w/2)),int(imgwidth+padding-(w/2)),stride):
        if img[j,i]>0.5:
          list_patches.append(1)
        else:
          list_patches.append(0)
    return list_patches

In [0]:
def load_image(fileName):
    return mpimg.imread(fileName)
  
def load_gray_image(infilename):
    img = cv2.imread(infilename)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    resized_image = cv2.resize(gray, (72,72)) 
    return gray
  
def grey_scale_conversion(img):
    new_img = np.zeros((img.shape[0],img.shape[1]))
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            if img[i,j,1] == 1.0:
                new_img[i,j] = 1
    return new_img
  
def grey_scale_conversion(img):
    new_img = np.zeros((img.shape[0],img.shape[1]))
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            if img[i,j,1] == 1.0:
                new_img[i,j] = 1
    return new_img
  

def pad_image(data, padding):
    """
    Extend the canvas of an image. Mirror boundary conditions are applied.
    """
    if len(data.shape) < 3:
        # Greyscale image (ground truth)
        data = np.lib.pad(data, ((padding, padding), (padding, padding)), 'reflect')
    else:
        # RGB image
        data = np.lib.pad(data, ((padding, padding), (padding, padding), (0,0)), 'reflect')
    return data
    
def img_crop_gt(im, w, h, stride):
    """ Crop an image into patches (this method is intended for ground truth images). """
    assert len(im.shape) == 2, 'Expected greyscale image.'
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    for i in range(0,imgheight,stride):
        for j in range(0,imgwidth,stride):
            im_patch = im[j:j+w, i:i+h]
            list_patches.append(im_patch)
    return list_patches
    
      
    
def create_patches_gt(X, patch_size, stride):
    img_patches = np.asarray([img_crop_gt(X[i], patch_size, patch_size, stride) for i in range(X.shape[0])])
    # Linearize list
    img_patches = img_patches.reshape(-1, img_patches.shape[2], img_patches.shape[3])
    return img_patches
    
def group_patches(patches, num_images):
    return patches.reshape(num_images, -1)

def extract_img_features(filename, stride):
    img = load_image(filename)
    img_patches = img_crop(img, patch_size, patch_size, stride, padding)
    X = np.asarray([img_patches[i] for i in range(len(img_patches))])
    return X

def mask_to_submission_strings(model, image_filename):
    """ Reads a single image and outputs the strings that should go into the submission file. """
    img_number = int(re.search(r"\d+", image_filename).group(0))
    Xi = load_image(image_filename)
    Xi = Xi.reshape(1, Xi.shape[0], Xi.shape[1], Xi.shape[2])
    Zi = model.classify(Xi)
    Zi = Zi.reshape(-1)
    patch_size = 16
    nb = 0
    print("Processing " + image_filename)
    for j in range(0, Xi.shape[2], patch_size):
        for i in range(0, Xi.shape[1], patch_size):
            label = int(Zi[nb])
            nb += 1
            yield("{:03d}_{}_{},{}".format(img_number, j, i, label))